In [1]:
%load_ext autoreload
%autoreload 2


import os
import time
import copy
import json
import openai
from dotenv import load_dotenv

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from method.models import generate_random_values, generate_static_values, generate_llm_values
from method.ours.utils import create_driver, get_xpath, interact_with_input
from method.ours.history import HistoryTable
from method.ours.feedback import get_global_feedback

In [4]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA'
GRAPH_EMBEDDING_METHOD = 'NODE2VEC'

URL = 'https://www.aa.com/homePage.do'
# 'https://www.jetblue.com/'
# 'https://www.united.com/en/us'
# 'https://www.aircanada.com/ca/en/aco/home.html'


def get_to_form(driver):
    try:
        driver.get(URL)
        
        '''
        # AC - My Bookings
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
        ).click()
        '''
        
        '''
        # AC - Multi-city
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
        ).click()
        '''
        
    except:
        print('timeout')


def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/MAIN[1]/DIV[1]/DIV[2]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]'
        ))
    )


def find_button():
    return None

In [5]:
METHOD = 'llm'

In [6]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [7]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')
form = find_form()
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [8]:
history_table = HistoryTable(
    url=URL,
    xpath=form_xpath
)

In [9]:
values = None
if METHOD == 'static':
    values = generate_static_values(driver, inputs)
elif METHOD == 'random':
    values = generate_random_values(driver, inputs)
else:
    values = generate_llm_values(form.get_attribute('outerHTML'), openai.api_key)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate testing values for a web form. When given a web form, you need to generate value combinations that can pass and fail when tried on the form. Your response should be in the following format:\n{\npassing: <a base passing combination for the form>,\nfailing: <a list of failing combination for the form>\n}\nYour response must be parsable by Python's json.loads function. Use \"id\" attribute to refer to the fields.\n"
  },
  {
    "role": "user",
    "content": "<form name=\"reservationFlightSearchForm\" id=\"reservationFlightSearchForm\" onsubmit=\"submitSearch(getCurrentSearch())\" method=\"post\" action=\"/booking/find-flights\" class=\"zeta\">\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<div class=\"row-form\" style=\"margin-top: -6px; margin-bottom: -5px;\">\n     <div class=\"span8\">\n        <ul role=\"radiogroup\" style=\"float:left;\">\n            <li class=\"customComponent\" role=\"presentation\">\n        

  Response: ChatCompletion(id='chatcmpl-8QLb1DfHQ5ZB1Nhywt3vMFsWlq5B8', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{"passing": {"flightRadio": "flight", "flightSearchForm.tripType.roundTrip": "roundTrip", "reservationFlightSearchForm.originAirport": "YVR", "reservationFlightSearchForm.destinationAirport": "JFK", "flightSearchForm.adultOrSeniorPassengerCount": "1", "aa-leavingOn": "12/25/2022", "aa-returningFrom": "12/30/2022", "fhServiceClass": "coach"}, "failing": [{"flightRadio": "", "flightSearchForm.tripType.roundTrip": "", "reservationFlightSearchForm.originAirport": "", "reservationFlightSearchForm.destinationAirport": "", "flightSearchForm.adultOrSeniorPassengerCount": "", "aa-leavingOn": "", "aa-returningFrom": "", "fhServiceClass": ""}, {"flightRadio": "flight", "flightSearchForm.tripType.roundTrip": "roundTrip", "reservationFlightSearchForm.originAirport": "123", "reservationFlightSearchForm.destinationAirport": "456", "flightSearchF

In [10]:
values

{'passing': {'flightRadio': 'flight',
  'flightSearchForm.tripType.roundTrip': 'roundTrip',
  'reservationFlightSearchForm.originAirport': 'YVR',
  'reservationFlightSearchForm.destinationAirport': 'JFK',
  'flightSearchForm.adultOrSeniorPassengerCount': '1',
  'aa-leavingOn': '12/25/2022',
  'aa-returningFrom': '12/30/2022',
  'fhServiceClass': 'coach'},
 'failing': [{'flightRadio': '',
   'flightSearchForm.tripType.roundTrip': '',
   'reservationFlightSearchForm.originAirport': '',
   'reservationFlightSearchForm.destinationAirport': '',
   'flightSearchForm.adultOrSeniorPassengerCount': '',
   'aa-leavingOn': '',
   'aa-returningFrom': '',
   'fhServiceClass': ''},
  {'flightRadio': 'flight',
   'flightSearchForm.tripType.roundTrip': 'roundTrip',
   'reservationFlightSearchForm.originAirport': '123',
   'reservationFlightSearchForm.destinationAirport': '456',
   'flightSearchForm.adultOrSeniorPassengerCount': '0',
   'aa-leavingOn': '12/25/2022',
   'aa-returningFrom': '12/30/2022',

In [13]:
def get_element(element_id):
    return driver.find_element(By.ID if METHOD == 'llm' else By.XPATH, element_id)


def send_values(elemend_ids, values):
    for element_id in elemend_ids:
        try:
            element = get_element(element_id)
        except:
            continue
        value = values[element_id]
        
        element_type = element.get_attribute('type') or 'text'
    
        if element_type in ['radio', 'checkbox'] or element.tag_name in ['button']:
            continue
        
        try:
            interact_with_input(element, value)
        except:
            pass

        
def submit_form(form):
    # submit = driver.find_element(By.XPATH, f'{form_xpath}//*[@type="submit"]')
    # driver.find_element(By.TAG_NAME, 'body').click()
    submit = find_button()
    
    for _ in range(3):
        try:
            interact_with_input(submit, True)
            time.sleep(0.5)
        except:
            break

In [16]:
passing_set = {
    element_id: values[element_id]['passing'] for element_id in values.keys()
}


for element_id, element_values in values.items():
    get_to_form(driver)
    time.sleep(1)
    
    try:
        element = get_element(element_id)
    except:
        continue
    element_type = element.get_attribute('type') or 'text'
    
    if element_type in ['radio', 'checkbox', 'submit']:
        continue
    
    # print(element)
    
    for failing_value in element_values['failing']:
        # try:
        get_to_form(driver)
        time.sleep(1)
    
        passing_copy = copy.copy(passing_set)
        passing_copy[element_id] = failing_value
        send_values(values.keys(), passing_copy)

        time.sleep(1)

        submit_form(form)

        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

        history_table.add(
            passing_copy,
            'base',
            global_feedback,
            driver.current_url
        )
        # except Exception as e:
        #     print(e)

KeyboardInterrupt: 

In [14]:
for i, v in enumerate(history_table.table['values']):
    jv = json.loads(v)
    for key, value in jv.items():
        if key not in history_table.table.columns:
            history_table.table[key] = None
        history_table.table.loc[i, key] = value

In [19]:
history_table.table

,values,variation_type,feedback,new_url,type,name,category_id,description,sale_information,purchase_information,sale_price,purchase_price,tax_ids
0,"{""type"": ""product"", ""name"": """", ""category_id"":...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,,5,This is a test product,sale,sale,100,80,
1,"{""type"": ""product"", ""name"": ""123"", ""category_i...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,123,5,This is a test product,sale,sale,100,80,
2,"{""type"": ""product"", ""name"": ""Test Product 1"", ...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product 1,5,This is a test product,sale,sale,100,80,
3,"{""type"": ""product"", ""name"": ""Test Product 2"", ...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product 2,5,This is a test product,sale,sale,100,80,
4,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,,This is a test product,sale,sale,100,80,
5,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,123,This is a test product,sale,sale,100,80,
6,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,General,This is a test product,sale,sale,100,80,
7,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,New Category,This is a test product,sale,sale,100,80,
8,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,5,,sale,sale,100,80,
9,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,5,123,sale,sale,100,80,
